In [1]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Polygon
from geojson import GeoJSON
import json
import matplotlib
from numpy import mean, all, median
from tqdm import tqdm_notebook as tqdm
tqdm().pandas()

from bokeh.models import (ColumnDataSource, GMapPlot, Range1d, ColumnDataSource, LinearAxis,
HoverTool, PanTool, WheelZoomTool, BoxZoomTool, ResetTool,CustomJS,
GMapOptions, NumeralTickFormatter, PrintfTickFormatter, MapOptions)
from bokeh.models.glyphs import Patches
from bokeh.plotting import figure, save, show
from bokeh.resources import CDN
from bokeh.io import output_file, show, saving
from bokeh.models.widgets import RadioButtonGroup
from bokeh.layouts import widgetbox, column
from bokeh.models.widgets import Panel, Tabs
outfp = r"./map_display.html"
from bokeh import tile_providers
from bokeh.tile_providers import CARTODBPOSITRON
import pyproj

df_wash = pd.read_pickle('Washington.pickle')
df_wash.head()

df = df_wash[df_wash['total_score'].apply(len)>0].copy()
# This will discard all the hexes where there aren't any observations.
# remember to make this a copy or else you will end up with warnings about writing to a slice
df.shape

HBox(children=(IntProgress(value=1, bar_style='info', max=1), HTML(value='')))

(5107, 9)

In [2]:
df['total_median'] = df.total_score.progress_apply(median)
df['total_mean'] = df.total_score.progress_apply(mean)

colors = [
'#ffffcc',
'#d9f0a3',
'#addd8e',
'#78c679',
'#31a354',
'#006837'
]
descriptions = ['Bad','Poor','Average','Good','Great','Excellent']
bins = [-1,50,60,70,80,90,100]
columns = ['total_mean']
metrics = [mean, median]

def summerizer(df):
    for col in columns:
        string1 = col + "_color"
        string2 = col + "_rating"
        df[string1] = pd.cut(df[col], bins, labels=colors)
        df[string2] = pd.cut(df[col], bins, labels=descriptions)


summerizer(df)        

# Okay, it only took me five times to realize I should put a backup point here.  Progress?
df.to_pickle("summerized.pickle")

HBox(children=(IntProgress(value=0, max=5107), HTML(value='')))

HBox(children=(IntProgress(value=0, max=5107), HTML(value='')))

In [3]:
df = pd.read_pickle('summerized.pickle')
df.head()

,grid,total_score,bike_score,car_score,mass_score,ride_score,points_list,Y,X,total_median,total_mean,total_mean_color,total_mean_rating
hex_key,,,,,,,,,,,,,
80603,"(2, 0, -2)",[100],[11.0],[5.0],[72.0],[12.0],[279253],"[4708165.371625002, 4708213.251571501, 4708213...","[-8575794.351447944, -8575821.994947944, -8575...",100.0,100.0,#006837,Excellent
81402,"(1, 2, -3)",[100],[16.0],[4.0],[68.0],[12.0],[241742],"[4708309.011464498, 4708356.891410997, 4708356...","[-8575877.281947942, -8575904.925447943, -8575...",100.0,100.0,#006837,Excellent
81800,"(-1, 3, -2)","[76, 88]","[18.0, 0.0]","[0.0, 4.0]","[55.0, 78.0]","[27.0, 18.0]","[226568, 241755]","[4708309.011464498, 4708356.891410997, 4708356...","[-8576043.142947944, -8576070.786447944, -8576...",82.0,82.0,#31a354,Great
79404,"(3, -3, 0)",[100],[8.0],[2.0],[80.0],[10.0],[242034],"[4707925.971892506, 4707973.851839005, 4707973...","[-8575711.420947943, -8575739.064447943, -8575...",100.0,100.0,#006837,Excellent
81798,"(-3, 3, 0)","[96, 97, 98]","[10.0, 10.0, 10.0]","[3.0, 3.0, 3.0]","[74.0, 74.0, 74.0]","[13.0, 13.0, 13.0]","[241278, 241284, 243140]","[4708213.251571501, 4708261.131518, 4708261.13...","[-8576209.003947943, -8576236.647447944, -8576...",97.0,97.0,#006837,Excellent


In [4]:
output_file(outfp)
dsource = ColumnDataSource(df)

wgs84=pyproj.Proj("+init=EPSG:4326") # LatLon with WGS84 datum used by GPS units and Google Earth
webdb=pyproj.Proj("+init=EPSG:3857") # Web Mercator

class plot_pipeline:
    def __init__(self, source, center):
        self.source = source
        self.make_fig(center[0],center[1])
    
    def make_fig(self, lat, lon):
        
        x, y = pyproj.transform(wgs84, webdb, lon, lat)
        
        tips = [
                ('Total Score', "@total_mean"),
                ('Description', "@total_mean_rating")
        ]
        wheel = WheelZoomTool()
        tool_box = [HoverTool(tooltips=tips), wheel, PanTool(),
                 BoxZoomTool(), ResetTool()]
        bound = 15000 # 15 km
        plot = figure(x_range=Range1d(x-bound,x+bound),
            y_range=Range1d(y-bound, y+bound),
            toolbar_location="right",
            tools = tool_box, plot_width=1100,
            plot_height=1100)
        
        plot.toolbar.active_scroll = wheel
        plot.add_tile(CARTODBPOSITRON)
        self.plot=plot
        
    def add_layer(self, column_name):       
        
        hexes = Patches(xs='X', ys='Y',
            fill_color=column_name, fill_alpha=0.4, 
            line_color="black", line_width=1.00)
        
        self.plot.add_glyph(dsource, hexes)
        
    def out(self):
        show(self.plot)
        
        
import pyproj
wgs84=pyproj.Proj("+init=EPSG:4326") # LatLon with WGS84 datum used by GPS units and Google Earth
webdb=pyproj.Proj("+init=EPSG:3857") # Web Mercator

washington_y = 38.904978
washington_x = -77.039658

        
wash = plot_pipeline(dsource, (washington_y, washington_x))
wash.add_layer('total_mean_color')
wash.out()



In [ ]:
# wheel = WheelZoomTool()

# tool_box = [wheel, PanTool(),HoverTool(),
#          BoxZoomTool(), ResetTool()]


# bound = 15000 
# fig = figure() , tools=tool_box)
# fig.axis.visible = False
# fig
# output_file("wash_plot_maps.html")
# show(fig)